## Load necessary Libraries

In [63]:
#Import Libraries
# Probably more functions than you'll need...
from bs4 import BeautifulSoup
import re
import pandas as pd
from spellchecker import SpellChecker
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import matplotlib
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import ast


import numpy as np
# Install new libraries
#%pip install pyspellchecker
#%pip install TextBlob

import warnings
warnings.filterwarnings('ignore')

## Load DataFrame from CSV


In [64]:
# Define File Path

data_path = 'pre_processed_vodafone_data.csv'

In [65]:
# Load processed dataset into DataFrame
preprocessed_vodafone_reviews = pd.read_csv(data_path)  


In [66]:
# Identify which columns are lists

columns = list(preprocessed_vodafone_reviews.columns)

# create empty list to contain a list of columns that are lists...  listslistslistslists...lists
list_columns = []


for column in columns:
    first_row_value = str(preprocessed_vodafone_reviews[column].iloc[0])
    first_character = first_row_value[0]
    last_character = first_row_value[len(first_row_value)-1]
    # A column /probably/ contains lists if values starte with a [ and end with a ]
    if first_character == "[" and last_character == "]":
        list_columns = list_columns + [column]
        
print(list_columns)

['sentences', 'noun_phrases', 'lemmatised_sentences', 'bi_grams', 'tri_grams', 'words', 'nouns']


In [67]:
# Lists will initially be expressed as strings when imported form CSV
# We will use AST to convert these string columns back to list
# This function could probably be better implemented without a for loop
# but works for now...

# List encode columns contain lists of tuples in the format:
# ("feature value", "Sentiment of originating sentence", "Review Promoter score")

# Abstract syntax tree is used to recreate the lists of tuples stored in the pre-processed data
for i in range(len(preprocessed_vodafone_reviews)):
    for column in list_columns:
        preprocessed_vodafone_reviews[column].iloc[i] = list(ast.literal_eval(preprocessed_vodafone_reviews[column].iloc[i]))

    
preprocessed_vodafone_reviews.head(5)

,review_id,title,review,score,timestamp,review_sentiment,sentences,lemmatised_title,noun_phrases,lemmatised_sentences,bi_grams,tri_grams,words,nouns
0,1,vodafone almost ruined my life - stolen identi...,my email and password were part of a company d...,1,2021-01-31 01:03:34+00:00,-0.9390,[(vodafone almost ruined my life - stolen iden...,vodafone almost ruin life steal identity security,"[(company database leak, -0.34, 1), (vodafone ...",[(vodafone almost ruin life steal identity sec...,"[(ruin life, -0.5584, 1), (life steal, -0.5584...","[(vodafone almost ruin, -0.5584, 1), (ruin lif...","[(vodafone, -0.5584, 1), (almost, -0.5584, 1),...","[(vodafone, -0.5584, 1), (life, -0.5584, 1), (..."
1,2,misguided sale,"misguided salecouple of days ago, i was told t...",1,2021-01-29 08:59:16+00:00,0.8555,"[(misguided sale, 0.0, 1), (misguided salecoup...",misguide sale,"[(trade in value, 0.34, 1), (old phone, 0.34, ...","[(misguide, 0.0, 1), (misguided salecouple day...","[(misguide sale, 0.0, 1), (misguided salecoupl...","[(misguided salecouple day, 0.34, 1), (trade i...","[(misguide, 0.0, 1), (misguided, 0.34, 1), (sa...","[(misguide, 0.0, 1), (salecouple, 0.34, 1), (d..."
2,3,worst network,worst network for data in australia. i used vo...,1,2021-01-28 12:16:46+00:00,-0.6858,"[(worst network, -0.6249, 1), (worst network f...",bad network,"[(mobile phone, -0.7227, 1), (i need data, -0....","[(bad network, -0.6249, 1), (bad network data ...","[(bad network, -0.6249, 1), (bad network, -0.6...","[(network for data, -0.6249, 1), (i use vodafo...","[(bad, -0.6249, 1), (network, -0.6249, 1), (ba...","[(network, -0.6249, 1), (network, -0.6249, 1),..."
3,4,insurance contract with them is bullshit,they charged me for over a year a plan with in...,1,2021-01-27 06:21:26+00:00,0.3612,"[(insurance contract with them is bullshit, -0...",insurance contract with them bullshit,"[(insurance contract, -0.5859, 1), (free phone...","[(insurance contract bullshit, -0.5859, 1), (c...","[(insurance contract, -0.5859, 1), (year plan,...","[(plan with insurance, 0.3612, 1), (month free...","[(insurance, -0.5859, 1), (contract, -0.5859, ...","[(insurance, -0.5859, 1), (contract, -0.5859, ..."
4,5,record all your phonecalls with them when you ...,my parents and i have been with vodafone for a...,1,2021-01-27 04:18:19+00:00,0.4926,[(record all your phonecalls with them when yo...,record all your phonecalls with them when you ...,"[(decent coverage, 0.0, 1), (cheap price, 0.0,...","[(record phonecalls issue, 0.0, 1), (parent vo...","[(decent coverage, 0.0, 1), (cheap price, 0.0,...","[(thats strictly due, 0.0, 1), (company with d...","[(record, 0.0, 1), (phonecalls, 0.0, 1), (issu...","[(record, 0.0, 1), (issue, 0.0, 1), (parent, 0..."


## Accessor Functions (And other functions)

In [68]:
# Returns a dataframe containing all occurances of a featiure value, 
# Along with the sentiment of the sentence in which they were found, 
# and the associated review score

# Pass the observation dataframe, and the column to extract
def term_df_with_scores(df, column):
    term_frame = pd.DataFrame(columns = ["term","sentiment","promoter_score"])

    for i in range(len(df)):
        row = df[column].iloc[i]
        row_frame = pd.DataFrame.from_records(row, columns=['term','sentiment','promoter_score'])
        term_frame = term_frame.append(row_frame)
            
    return term_frame


term_df_with_scores(preprocessed_vodafone_reviews, 'words').head()

,term,sentiment,promoter_score
0,vodafone,-0.5584,1
1,almost,-0.5584,1
2,ruin,-0.5584,1
3,life,-0.5584,1
4,steal,-0.5584,1


In [69]:
## Create a function to return the pearson correlation between score and sentiment 
# for Features encoded as leists of tuples

# Pass the observation dataframe and feature column to process
def get_corr(df, column):
    term_frame = term_df_with_scores(df, column)
    term_frame[["sentiment", "promoter_score"]] = term_frame[["sentiment", "promoter_score"]].apply(pd.to_numeric)
    result_frame = pd.DataFrame(columns = ["term","sentiment_correlation","count"])
    unique_terms = term_frame.term.unique()
    
    for term in unique_terms:
        temp_frame = term_frame[term_frame.term==term]
        correlation = temp_frame["sentiment"].corr(temp_frame["promoter_score"])
        count = len(temp_frame)
        if not np.isnan(correlation):
            result_frame = result_frame.append(pd.DataFrame([[term, correlation, count]], columns = ["term","sentiment_correlation","count"]))
    return result_frame

get_corr(preprocessed_vodafone_reviews, 'noun_phrases').head()

,term,sentiment_correlation,count
0,old phone,0.130361,14
0,new phone i,0.532814,4
0,mobile phone,0.352996,45
0,home wifi,1.000000,2
0,sure i,0.435640,11


In [70]:
# Get count of documents each word or phrase appears in
# As well as percentage, and rank

def doc_count(df, column):
    # Produce a 2 column df with the document ID and the column to be measured
    temp_df = pd.DataFrame(columns=['review_id', 'term'])
    num_reviews = df.review_id.nunique()
    for i in range(len(df)):
        review_id = df['review_id'].iloc[i]
        for item in df[column].iloc[i]:
            term = item[0]
            temp_df = temp_df.append(pd.DataFrame([[review_id, term]], columns = ["review_id", "term"]))
            
    results = temp_df.groupby(by='term', as_index=False).agg({'review_id': pd.Series.nunique})
    results['percentage'] = results['review_id']/num_reviews * 100
    results = results.sort_values(by='percentage',ascending=False)
    # add a column for rank
    results['rank'] = 0
    r = 1
    for i in range(len(results)):
        results['rank'].iloc[i] = r
        r += 1
    
    results = results.rename(columns={"review_id": "reviews_containing_term"})
    return results
    
    
doc_count(preprocessed_vodafone_reviews, 'noun_phrases').head()

,term,reviews_containing_term,percentage,rank
1651,customer service,368,19.337888,1
6994,time i,73,3.836048,2
4553,new phone,68,3.573305,3
4428,network coverage,48,2.522333,4
3884,loyal customer,45,2.364687,5


In [71]:
# return a dataframe containing all unique terms for
# a list encoded feature column
def get_unique_terms(df, column):
    
    temp_df = pd.DataFrame(columns=['term'])
    for i in range(len(df)):
        for item in df[column].iloc[i]:
            term = item[0]
            temp_df = temp_df.append(pd.DataFrame([[term]], columns = ["term"]))
            
    return pd.DataFrame(temp_df.term.unique(), columns=['term'])

get_unique_terms(preprocessed_vodafone_reviews, 'noun_phrases').head()

,term
0,company database leak
1,vodafone account
2,factor authentication
3,friday afternoon
4,two factor authentication


In [72]:
# For revoews containing each term, return the percentage
# of the reviews that are Detractors, Promoters, and  passive customers
# as well as the difference in count between
# detractors and promoters

def track_rank_changes(df, column):
    #results = pd.DataFrame(columns=['term', 'perc_detractor', 'perc_passive', 'perc_promoter', 'total_change'])
    all_unique_terms = get_unique_terms(df, column)
    #print(all_unique_terms)
    all_unique_terms['perc_detractor'] = 0
    all_unique_terms['perc_passive'] = 0
    all_unique_terms['perc_promoter'] = 0
    all_unique_terms['total_change'] = 0    
    
    
    detractors = doc_count(df[df['score'] <=3], column)
    passive = doc_count(df[df['score'] ==4], column)
    promotors = doc_count(df[df['score'] ==5], column)
    
    for i in range(len(all_unique_terms)):
        if all_unique_terms['term'].iloc[i] in detractors['term'].tolist():
            perc_detractor = detractors[detractors['term'] == all_unique_terms['term'].iloc[i]].percentage.iloc[0]
        else:
            perc_detractor = 0
            
        if all_unique_terms['term'].iloc[i] in passive['term'].tolist():
            perc_passive = passive[passive['term'] == all_unique_terms['term'].iloc[i]].percentage.iloc[0]
        else:
            perc_passive = 0
        
        if all_unique_terms['term'].iloc[i] in promotors['term'].tolist():
            perc_promoter = promotors[promotors['term'] == all_unique_terms['term'].iloc[i]].percentage.iloc[0]
        else:
            perc_promoter = 0
            
        total_change = perc_promoter - perc_detractor
        
        all_unique_terms['perc_detractor'].iloc[i] = perc_detractor
        all_unique_terms['perc_passive'].iloc[i] = perc_passive
        all_unique_terms['perc_promoter'].iloc[i] = perc_promoter
        all_unique_terms['total_change'].iloc[i] = total_change        

    return all_unique_terms.sort_values(by='total_change',ascending=False)




track_rank_changes(preprocessed_vodafone_reviews, 'noun_phrases').head()

,term,perc_detractor,perc_passive,perc_promoter,total_change
102,excellent customer service,0.000000,0.000000,5.844156,5.844156
371,excellent service,0.000000,0.000000,5.844156,5.844156
93,new phone,3.179364,4.878049,7.142857,3.963493
3393,great customer service,0.119976,0.000000,3.896104,3.776128
941,great service,0.179964,3.658537,3.896104,3.716140
